# Взаимодействие с БД

## 1. PSYCOPG

## Базовое применение

In [54]:
import psycopg2
dbname='storefront'
user='storefront'
password='storefront'
host='127.0.0.1'
port=5437

db_connection_args = dict(
    dbname=dbname, 
    user=user, 
    password=password,
    host=host,
    port=port,
)

In [29]:
# create connection
conn = psycopg2.connect(**db_connection_args)

In [30]:
# Open a cursor to perform database operations
cur = conn.cursor()

In [31]:
# Execute a command
cur.execute("SELECT * FROM student")

In [32]:
cur.fetchone()

(1, 'Петр', 'Петров', 'Петрович', datetime.date(2006, 1, 1), 525677889911, 1)

In [33]:
a = cur.fetchall()

In [34]:
a[:1]

[(2, 'Иван', 'Иванов', 'Иванович', datetime.date(2006, 2, 2), 525677889912, 1)]

In [35]:
len(a)

29

In [36]:
# finally close cursor
cur.close()

In [37]:
# finnaly close connection
conn.close()

## Стандартное применение

In [39]:
# exiting the connection’s with block doesn’t close the connection, but only the transaction associated to it
with psycopg2.connect(**db_connection_args) as conn:
    with conn.cursor() as cur:
        cur.execute("SELECT * FROM student")

conn.close()

In [40]:
cur.fetchone()

InterfaceError: cursor already closed

In [41]:
# exiting the connection’s with block doesn’t close the connection, but only the transaction associated to it
with psycopg2.connect(**db_connection_args) as conn:
    with conn.cursor() as cur:
        cur.execute("SELECT * FROM student")
        result = cur.fetchall()

conn.close()

In [42]:
result[:1]

[(1, 'Петр', 'Петров', 'Петрович', datetime.date(2006, 1, 1), 525677889911, 1)]

In [45]:
# exiting the connection’s with block doesn’t close the connection, but only the transaction associated to it
with psycopg2.connect(**db_connection_args) as conn:
    with conn.cursor() as cur:
        cur.execute("SELECT count(*) FROM student")
        result = cur.fetchone()

conn.close()

In [46]:
result

(30,)

## 2. Pandas

In [47]:
import pandas as pd

### Прочитать из БД в датафрейм

In [51]:
with psycopg2.connect(**db_connection_args) as conn:
    df = pd.read_sql('select * from student', con=conn)

conn.close()

df.head()

/tmp/ipykernel_7860/849527893.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql('select * from student', con=conn)


,id,first_name,last_name,middle_name,date_of_birth,inn,groups_id
0,1,Петр,Петров,Петрович,2006-01-01,525677889911,1
1,2,Иван,Иванов,Иванович,2006-02-02,525677889912,1
2,3,Никита,Никитин,Олегович,2006-03-03,525677889913,1
3,4,Марина,Маринина,Ивановна,2006-03-04,525677889914,1
4,5,Жанна,Сергеева,Алексеевна,2006-01-01,525677889915,1


### Положить в БД из датафрейма

In [53]:
!pip install sqlalchemy

  Using cached greenlet-3.1.1-cp310-cp310-manylinux_2_24_x86_64.manylinux_2_28_x86_64.whl.metadata (3.8 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 4.4 MB/s eta 0:00:00m eta 0:00:010:00:01
Using cached greenlet-3.1.1-cp310-cp310-manylinux_2_24_x86_64.manylinux_2_28_x86_64.whl (599 kB)

[notice] A new release of pip is available: 23.3.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [56]:
from sqlalchemy import create_engine

engine = create_engine(f'postgresql://{user}:{password}@{host}:{port}/{dbname}')

# name = table name
df.to_sql(name='test_data', con=engine)

conn.close()